# Broadcasting and Vectorization

Learn how NumPy efficiently handles operations on arrays of different shapes.

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand and apply broadcasting rules
2. Write vectorized code instead of loops
3. Compare performance of vectorized vs loop-based code
4. Use np.newaxis for dimension manipulation

In [ ]:
import numpy as np

---

## 1. What is Broadcasting?

Broadcasting allows NumPy to work with arrays of different shapes during arithmetic operations.

In [ ]:
# Scalar broadcast
arr = np.array([1, 2, 3, 4, 5])
result = arr * 2  # 2 is broadcast to [2, 2, 2, 2, 2]

print(f"arr: {arr}")
print(f"arr * 2: {result}")

In [ ]:
# 1D array with 2D array
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])
row = np.array([10, 20, 30])

print(f"Matrix:\n{matrix}")
print(f"\nRow: {row}")
print(f"\nMatrix + Row:\n{matrix + row}")

### Broadcasting Rules

1. If arrays have different number of dimensions, pad the smaller shape with 1s on the left
2. Arrays with size 1 in a dimension act as if they had the size of the largest array in that dimension
3. Arrays are compatible if dimensions are equal OR one of them is 1

In [ ]:
# Rule demonstration
a = np.ones((3, 4))
b = np.ones((4,))

print(f"a.shape: {a.shape}")
print(f"b.shape: {b.shape}")

# Step 1: b becomes (1, 4)
# Step 2: b stretched to (3, 4)
result = a + b
print(f"(a + b).shape: {result.shape}")

In [ ]:
# More examples
a = np.arange(3).reshape(3, 1)  # Shape (3, 1)
b = np.arange(4)                 # Shape (4,)

print(f"a (3, 1):\n{a}")
print(f"\nb (4,): {b}")

# a: (3, 1) -> compatible
# b: (4,) -> (1, 4) -> compatible
# Result: (3, 4)
result = a + b
print(f"\na + b (3, 4):\n{result}")

In [ ]:
# Incompatible shapes
a = np.ones((3, 4))
b = np.ones((3,))

print(f"a.shape: {a.shape}")
print(f"b.shape: {b.shape}")

# b becomes (1, 3) but 3 != 4 and neither is 1
try:
    result = a + b
except ValueError as e:
    print(f"Error: {e}")

---

## 2. Using np.newaxis

In [ ]:
# np.newaxis adds a dimension
arr = np.array([1, 2, 3])  # Shape (3,)

row = arr[np.newaxis, :]  # Shape (1, 3)
col = arr[:, np.newaxis]  # Shape (3, 1)

print(f"Original shape: {arr.shape}")
print(f"Row vector shape: {row.shape}")
print(f"Column vector shape: {col.shape}")
print(f"\nRow:\n{row}")
print(f"\nColumn:\n{col}")

In [ ]:
# Create outer product using broadcasting
a = np.array([1, 2, 3])
b = np.array([10, 20, 30, 40])

outer = a[:, np.newaxis] * b[np.newaxis, :]
# Also: outer = np.outer(a, b)

print(f"a: {a}")
print(f"b: {b}")
print(f"\nOuter product:\n{outer}")

---

## 3. Practical Broadcasting Examples

In [ ]:
# Centering data (subtract column means)
data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9],
                 [10, 11, 12]])

col_means = data.mean(axis=0)  # Shape (3,)
centered = data - col_means    # Broadcasting!

print(f"Original:\n{data}")
print(f"\nColumn means: {col_means}")
print(f"\nCentered (column means = 0):\n{centered}")
print(f"\nVerify: {centered.mean(axis=0)}")

In [ ]:
# Standardization (z-scores)
col_means = data.mean(axis=0)
col_stds = data.std(axis=0)
standardized = (data - col_means) / col_stds

print(f"Standardized:\n{standardized}")
print(f"\nNew means: {standardized.mean(axis=0)}")
print(f"New stds: {standardized.std(axis=0)}")

In [ ]:
# Distance calculation (Euclidean)
points = np.array([[0, 0],
                   [1, 1],
                   [2, 0]])
reference = np.array([0, 0])

# Distance from each point to reference
diff = points - reference  # Broadcasting
distances = np.sqrt(np.sum(diff**2, axis=1))

print(f"Points:\n{points}")
print(f"Reference: {reference}")
print(f"Distances: {distances}")

In [ ]:
# Pairwise distances (all points to all points)
points = np.array([[0, 0],
                   [1, 1],
                   [2, 0]])

# Create difference matrix using broadcasting
diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
# Shape: (3, 1, 2) - (1, 3, 2) = (3, 3, 2)

pairwise_dist = np.sqrt(np.sum(diff**2, axis=2))

print(f"Pairwise distances:\n{pairwise_dist}")

---

## 4. Vectorization vs Loops

In [ ]:
# Example: element-wise square
arr = np.arange(1000000)

# Loop approach
def square_loop(arr):
    result = np.empty_like(arr)
    for i in range(len(arr)):
        result[i] = arr[i] ** 2
    return result

# Vectorized approach
def square_vectorized(arr):
    return arr ** 2

# Compare timing
%timeit square_loop(arr)
%timeit square_vectorized(arr)

In [ ]:
# Example: sum of products
a = np.random.rand(10000)
b = np.random.rand(10000)

# Loop
def dot_loop(a, b):
    total = 0
    for i in range(len(a)):
        total += a[i] * b[i]
    return total

# Vectorized
def dot_vectorized(a, b):
    return np.sum(a * b)

# Even better: np.dot
def dot_numpy(a, b):
    return np.dot(a, b)

%timeit dot_loop(a, b)
%timeit dot_vectorized(a, b)
%timeit dot_numpy(a, b)

In [ ]:
# Example: conditional replacement
arr = np.random.randn(100000)

# Loop
def replace_negative_loop(arr):
    result = arr.copy()
    for i in range(len(result)):
        if result[i] < 0:
            result[i] = 0
    return result

# Vectorized
def replace_negative_vectorized(arr):
    result = arr.copy()
    result[result < 0] = 0
    return result

# np.where
def replace_negative_where(arr):
    return np.where(arr < 0, 0, arr)

# np.clip
def replace_negative_clip(arr):
    return np.clip(arr, 0, None)

%timeit replace_negative_loop(arr)
%timeit replace_negative_vectorized(arr)
%timeit replace_negative_where(arr)
%timeit replace_negative_clip(arr)

---

## 5. Common Vectorization Patterns

In [ ]:
# Pattern 1: Apply function to all elements
arr = np.array([1, 4, 9, 16, 25])

# Instead of: [np.sqrt(x) for x in arr]
result = np.sqrt(arr)
print(f"Square roots: {result}")

In [ ]:
# Pattern 2: Conditional operations
arr = np.array([-2, -1, 0, 1, 2])

# Instead of loop with if/else
result = np.where(arr > 0, arr, 0)  # ReLU
print(f"ReLU: {result}")

result = np.sign(arr)  # -1, 0, or 1
print(f"Sign: {result}")

In [ ]:
# Pattern 3: Aggregation with condition
arr = np.array([1, -2, 3, -4, 5, -6])

# Sum of positive values
positive_sum = np.sum(arr[arr > 0])
print(f"Sum of positive: {positive_sum}")

# Count of negative values
negative_count = np.sum(arr < 0)
print(f"Count of negative: {negative_count}")

In [ ]:
# Pattern 4: Row/column operations
matrix = np.array([[1, 2, 3],
                   [4, 5, 6]])

# Normalize rows (each row sums to 1)
row_sums = matrix.sum(axis=1, keepdims=True)
normalized = matrix / row_sums

print(f"Original:\n{matrix}")
print(f"\nRow normalized:\n{normalized}")
print(f"Row sums: {normalized.sum(axis=1)}")

---

## Exercises

### Exercise 1: Broadcasting Practice

Given a 4x3 matrix and a 1D array of length 3:
1. Add the 1D array to each row
2. Multiply each column by the 1D array (you'll need reshaping)
3. Create a 4x3 matrix where element [i,j] = row[j] * col[i] using broadcasting

In [ ]:
matrix = np.ones((4, 3))
row = np.array([1, 2, 3])
col = np.array([10, 20, 30, 40])
# Your code here


### Exercise 2: Vectorize This Loop

Replace the following loop with vectorized NumPy operations.

In [ ]:
# Original loop
arr = np.random.randn(1000)
result = []
for x in arr:
    if x > 0:
        result.append(x ** 2)
    else:
        result.append(0)
result = np.array(result)

# Your vectorized version:


### Exercise 3: Distance Matrix

Given 5 points in 2D, calculate the pairwise distance matrix (5x5) using broadcasting.

In [ ]:
points = np.array([[0, 0],
                   [1, 0],
                   [0, 1],
                   [1, 1],
                   [2, 2]])
# Your code here


---

## Solutions

<details>
<summary>Click to reveal Exercise 1 solution</summary>

```python
matrix = np.ones((4, 3))
row = np.array([1, 2, 3])
col = np.array([10, 20, 30, 40])

# 1. Add row to each row
result1 = matrix + row
print(f"Add row:\n{result1}")

# 2. Multiply each column
result2 = matrix * col[:, np.newaxis]  # or col.reshape(-1, 1)
print(f"\nMultiply column:\n{result2}")

# 3. Outer product style
result3 = row[np.newaxis, :] * col[:, np.newaxis]
print(f"\nOuter product:\n{result3}")
```

</details>

<details>
<summary>Click to reveal Exercise 2 solution</summary>

```python
arr = np.random.randn(1000)

# Vectorized version 1: boolean indexing
result = np.zeros_like(arr)
positive = arr > 0
result[positive] = arr[positive] ** 2

# Vectorized version 2: np.where
result = np.where(arr > 0, arr**2, 0)

print(f"Result: {result[:10]}")
```

</details>

<details>
<summary>Click to reveal Exercise 3 solution</summary>

```python
points = np.array([[0, 0],
                   [1, 0],
                   [0, 1],
                   [1, 1],
                   [2, 2]])

# Using broadcasting
diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
distances = np.sqrt(np.sum(diff**2, axis=2))

print(f"Distance matrix:\n{distances}")
```

</details>

---

## Summary

In this notebook, you learned:

- **Broadcasting rules** for operating on arrays of different shapes
- **np.newaxis** to add dimensions for broadcasting
- **Vectorization** is much faster than Python loops
- Common patterns: conditional operations, aggregations, normalization
- Use `%timeit` to compare performance

---

## Next Steps

Continue to [05_linear_algebra_random.ipynb](05_linear_algebra_random.ipynb) to learn about matrix operations and random number generation.